Turn chicken scratches into module.

Reference:
-http://www.cnccookbook.com/CCCNCGCodeG20G21MetricImperialUnitConversion.htm
- http://marlinfw.org/meta/gcode/
- https://github.com/grbl/grbl/wiki
- http://www.linuxcnc.org/docs/2.5/html/gcode/other-code.html

In [3]:
%load_ext autoreload
%autoreload 1

#### V0.07

In [4]:
%%file GCode.py
class GCode(object):
    def __init__(self, machine = None, buffer=None):
        self.linenumbering = False
        self.machine = machine
        if buffer is None:
            self.buffer = list()
        else:
            self.buffer = buffer
    @property
    def code(self):
        if self.linenumbering:
            pass
        return "\n".join(self.buffer)
    
    def load(self, filename):
        with open(filename, "r") as fid:
            data = fid.read()
        self.buffer = data.splitlines()
        
    def save(self, filename):
        with open(filename, "w") as fid:
            print(str(self), file=fid)
    
    def __str__(self):
        return self.code

    def __repr__(self):
        return "<GCode>[cmds={}]".format(len(self.buffer))
    
    def _repr_html_(self):
        html = list()
        for cmd_line in self.buffer:
            cmd, *args = cmd_line.split(" ")
            html_line = "<b>{cmd}</b> <i>{args}</i>".format(cmd=cmd, args=" ".join(args))
            html.append(html_line)
        return "<br>\n".join(html)
    
    def __add__(self, other):
        buffer = self.buffer
        buffer2 = other.buffer
        
        buffer.extend(buffer2)
        
        return GCode(machine=self.machine, buffer=buffer)
        
    def __iter__(self):
        self.line = -1
        return self
    
    def run(self):
        if self.machine is None:
            raise Exception("No machine to run on")
        self.machine.run(self)
    
    def __next__(self):
        self.line += 1
        if self.line >= len(self.buffer):
            raise StopIteration
        return self.buffer[self.line]
    
    def optimise(self):
        """ Optimize the GCode """
        raise(NotImplemented(""))

def cmd_factory(cmd, doc=None):
    def cmd_fcn(self, **kwargs):
        args = list()
        for key, value in kwargs.items():
            args.append("{key}{value}".format(key=key, value=value))
        
        cmd_str = "{cmd} {args}".format(cmd=cmd, args=" ".join(args))
        # For commands with no arguments.
        cmd_str = cmd_str.strip()
        self.buffer.append(cmd_str)  
    return cmd_fcn      

# Good core to start with.
commands=list()
# GCodes
for code in [0, 1, 2, 3, 4, 20, 21, 28, 90, 91]:
    commands.append("G{code}".format(code=code))
# MCodes
for code in [0, 1, 2, 3, 4, 5, 6]:
    commands.append("M{code}".format(code=code))

for command in commands:
    setattr(GCode, command, cmd_factory(command))

Overwriting GCode.py


In [5]:
%aimport GCode

In [6]:
square_20 = GCode.GCode()

In [7]:
square_20.load("square_20mm.gcode")

In [8]:
square_20

<GCode>[cmds=9]

In [9]:
from GRBL import GRBL

In [10]:
cnc = GRBL(port="/dev/cnc_3018")

In [11]:
square_20 = GCode.GCode(machine=cnc)

In [ ]:
square_20.load("square_20mm.gcode")

In [ ]:
square_20.run() # It works...

def metric

In [ ]:
def init(feed = 200, laser = 25):
    program = GCode.GCode()
    program.G21() # Metric Units
    program.G91() # Absolute positioning.
    program.G1(F=feed) #
    program.M4(S=laser) # Laser settings.
    return program

In [ ]:
def end():
    program = GCode.GCode()
    program.M5() # Laser settings.
    return program

In [ ]:
def square(size=20):    
    program = GCode.GCode()
    program.G1(X=size)
    program.G1(Y=size)
    program.G1(X=-size)
    program.G1(X=-size)
    return program

In [ ]:
init() + square() + end()

In [ ]:
cnc.run(init() + square() + end())

In [ ]:
cnc.cmd("G0 Y-90")

In [ ]:
cnc.run(init(laser=50) + square() + end())

In [ ]:
cnc.run(init(laser=100) + square() + end())

In [ ]:
def init(feed = 200, laser = 25):
    program = GCode.GCode()
    program.G21() # Metric Units
    program.G91() # Absolute positioning.
    program.G1(F=feed) #
    program.M3(S=laser) # Laser settings.
    return program

In [ ]:
cnc.run(init(laser=1))

In [ ]:
cnc.run(init(laser=75) + square() + end())

In [ ]:
init(laser=75) + square() + end()